### Ввод данных
Здесь задаются все переменные, касающиеся сравниваемых условий эксперимента:

In [1]:
# С этим префиксом будут записываться промежуточные и итоговые файлы
name <- '3vs13'

# По этому пути будут искаться файлы cov.gz (выходы Bismark):
working_dir <- "D:/current_work/POLY/НИР/bismark/coverage files" 

# Названия файлов cov.gz - сначала все репликаты одной группы,
# затем все репликаты другой (по 3 в каждой). Групп две - эксперимент и контроль:
samples = c(
    "n3_1_1_bismark_bt2_pe.deduplicated.bismark.cov.gz",
    "n3_2_1_bismark_bt2_pe.deduplicated.bismark.cov.gz",
    "n3_3_1_bismark_bt2_pe.deduplicated.bismark.cov.gz",
    
    "n13_1_1_bismark_bt2_pe.deduplicated.bismark.cov.gz",
    "n13_2_1_bismark_bt2_pe.deduplicated.bismark.cov.gz",
    "n13_3_1_bismark_bt2_pe.deduplicated.bismark.cov.gz"
    )

# В том же порядке короткие имена каждого из репликатов:
colnames <- c('n3_1','n3_2','n3_3','n13_1', 'n13_2', 'n13_3')

---------------------------------------------------------------------------------------------------------------

In [2]:
R.Version()$version.string
library(bsseq)

[1] "R version 4.2.1 (2022-06-23 ucrt)"

Загрузка требуемого пакета: BiocGenerics


Присоединяю пакет: 'BiocGenerics'


Следующие объекты скрыты от 'package:stats':

    IQR, mad, sd, var, xtabs


Следующие объекты скрыты от 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Загрузка требуемого пакета: GenomicRanges

Загрузка требуемого пакета: stats4

Загрузка требуемого пакета: S4Vectors


Присоединяю пакет: 'S4Vectors'


Следующие объекты скрыты от 'package:base':

    expand.grid, I, unname


Загрузка требуемого пакета: IRanges


Присоединяю пакет: 'IRanges'


Следующий объект скрыт от 'package:grDevices':

    windows


Загрузка требуемого пакета: GenomeInfoDb

Загруз

Сделаем рабочей директорию с cov-файлами, которые получили в результате работы Bismark:

In [3]:
setwd(working_dir)

В этом исследовании мы работаем с данными бисульфитного секвенирования двух диких сортов, одного элитного сорта и их потомков:

| BS  | WGS   | Accession name | Тип генотипа |
| --- |:-----:|:--------------:|-------------:|
| n1  | d_1   | TR83005        | Дикий вид, Kalkan_064 |
| n3  | d_3   | TR83052        | Дикий вид , Derei_070 |
| n4  | n-308 | ICCV96029      | Элитный сорт |
| n9  | n-196 | 630062         | ICCV96029_x_Kalka064_01859_F2 |
| n10 | n-206 | 630072         | ICCV96029_x_Kalka064_01905_F2 |
| n11 | n-125 | 629993         | ICCV96029_x_Derei070_01239_F2 |
| n12 | n-128 | 629996         | ICCV96029_x_Derei070_01244_F2 |
| n13 | n-129 | 629997         | ICCV96029_x_Derei070_01244_F2 |
| n14 | n-130 | 629998         | ICCV96029_x_Derei070_01250_F2 |
| n17 | n-200 | 630066         | ICCV96029_x_Kalka064_01877_F2 |


Для каждого сорта имеем по три биологических репликата, что обозначается так: n<номер сорта>_<номер репликата>. То есть, n4_2 соответствует второму репликату сорта 4, который, как видим по табличке сверху, является элитным.

Замечу, что постфикс _1 в названии выходных файлов Bismark не имеет смысловой нагрузки, такое имя было выбрано программой автоматически при обработке парных ридов.

Многие операции выполняются долго, поэтому для более быстрого страрта при перезапуске jupyter notebook'а я сохраняю промежуточные результаты в бинарные файлы с расширением .rds, их можно потом прочитать прямо стурктуру данных языка R, аналогичную той, из которой они были созданы.

После записи промежуточных файлов код клеток с долго выполняющимися операциями может быть закомментирован. Если Вы хотите перезаписать запомненные файлы, то раскомментируйте код, выделив и нажав ctrl+/, и запустите её. Такие клетки я помечу "TIME CONSUMING"

# Загружаем bismark.cov.gz данные о покрытии для обработки библиотекой bsseq

In [4]:
# TIME CONSUMING

bsseq <- bsseq::read.bismark(samples)
colnames(bsseq) <- colnames

saveRDS(bsseq, paste(name, "_bsseq.rds", sep="")) 

In [4]:
bsseq = readRDS(paste(name, "_bsseq.rds", sep=""))

# Поиск DMR с использованием сглаживающего алгоритма BSmooth в bsseq

In [6]:
# TIME CONSUMING

smoothed_bsseq = BSmooth(bsseq, h=2000)

saveRDS(smoothed_bsseq, paste(name, "_smoothed_bsseq.rds", sep=""))

In [5]:
smoothed_bsseq = readRDS(paste(name, "_smoothed_bsseq.rds", sep=""))

In [6]:
bsseq_cov <- getCoverage(smoothed_bsseq)

In [7]:
Type <- c("group1", "group1", "group1", "group2", "group2", "group2")
df <- data.frame(Type)
row.names(df) <- colnames
pData(bsseq) <- df

In [8]:
loci_to_keep <- which(rowSums(bsseq_cov[, bsseq$Type == "group1"] >= 2) >= 2 &
                     rowSums(bsseq_cov[, bsseq$Type == "group2"] >= 2) >= 2)

In [9]:
length(loci_to_keep)
length(bsseq)

[1] 6222538

[1] 8422253

In [10]:
smoothed_bsseq <- smoothed_bsseq[loci_to_keep,]

In [11]:
t_test_smoothed_bsseq = BSmooth.tstat(smoothed_bsseq, colnames[c(1, 2, 3)], colnames[c(4, 5, 6)])
stats_t_test = getStats(t_test_smoothed_bsseq)

[BSmooth.tstat] preprocessing ... done in 8.4 sec
[BSmooth.tstat] computing stats within groups ... done in 5.6 sec
[BSmooth.tstat] computing stats across groups ... done in 18.0 sec


In [12]:
ps <- getStats(t_test_smoothed_bsseq)[,"tstat.corrected"]

DMRs = dmrFinder(t_test_smoothed_bsseq,cutoff = c(quantile(ps, 0.005), quantile(ps, 0.995)))

[dmrFinder] creating dmr data.frame


We rank DMRs by the column areaStat which is the sum of the t-statistics in each CpG. This is kind of the area of the DMR, except that it is weighted by the number of CpGs and not by genomic length. This is currently the best statistic we know, although it is far from perfect (we would like to do something better).

Here, we filter out DMRs that do not have at least 3 CpGs in them and at least a mean difference (across the DMR) in methylation between normal and cancers of at least 0.1. While the exact values of these two filters can be debated, it is surely a good idea to use something like this.


In [13]:
DMRs <- subset(DMRs, n >= 3 & abs(meanDiff) >= 0.1)
dim(DMRs)

[1] 5465   16

In [16]:
pData <- pData(bsseq)
pData$col <- rep(c("red", "blue"), each = 3)
pData(bsseq) <- pData
pData(bsseq)

DataFrame with 6 rows and 2 columns
             Type         col
      <character> <character>
n3_1       group1         red
n3_2       group1         red
n3_3       group1         red
n13_1      group2        blue
n13_2      group2        blue
n13_3      group2        blue

In [17]:
#range.dmr = data.frame(DMRs$start, DMRs$end, DMRs$chr)
#names(range.dmr) = c("start", "end", "chr")
pdf(file=paste(name, "_dmrs_top10.pdf", sep=""), width = 10, height = 5)
par(ask=T)

plotManyRegions(smoothed_bsseq, DMRs[1:10,], col=pData$col, extend = 5000, addRegions = DMRs)
dev.off()

[plotManyRegions] preprocessing ...done
[plotManyRegions]   plotting region 1 (out of 10)
[plotManyRegions]   plotting region 2 (out of 10)
[plotManyRegions]   plotting region 3 (out of 10)
[plotManyRegions]   plotting region 4 (out of 10)
[plotManyRegions]   plotting region 5 (out of 10)
[plotManyRegions]   plotting region 6 (out of 10)
[plotManyRegions]   plotting region 7 (out of 10)
[plotManyRegions]   plotting region 8 (out of 10)
[plotManyRegions]   plotting region 9 (out of 10)
[plotManyRegions]   plotting region 10 (out of 10)


png 
  2

In [18]:
DMRs_filtered_by_width <- DMRs[DMRs$width > quantile(DMRs$width, 0.995), ]
dim(DMRs_filtered_by_width)

[1] 28 16

In [19]:
pdf(file=paste(name, "_dmrs_top_wide.pdf", sep=""), width = 10, height = 5)
par(ask=T)

plotManyRegions(smoothed_bsseq, DMRs_filtered_by_width, col=pData$col, extend = 5000, addRegions = DMRs)
dev.off()

[plotManyRegions] preprocessing ...done
[plotManyRegions]   plotting region 1 (out of 28)
[plotManyRegions]   plotting region 2 (out of 28)
[plotManyRegions]   plotting region 3 (out of 28)
[plotManyRegions]   plotting region 4 (out of 28)
[plotManyRegions]   plotting region 5 (out of 28)
[plotManyRegions]   plotting region 6 (out of 28)
[plotManyRegions]   plotting region 7 (out of 28)
[plotManyRegions]   plotting region 8 (out of 28)
[plotManyRegions]   plotting region 9 (out of 28)
[plotManyRegions]   plotting region 10 (out of 28)
[plotManyRegions]   plotting region 11 (out of 28)
[plotManyRegions]   plotting region 12 (out of 28)
[plotManyRegions]   plotting region 13 (out of 28)
[plotManyRegions]   plotting region 14 (out of 28)
[plotManyRegions]   plotting region 15 (out of 28)
[plotManyRegions]   plotting region 16 (out of 28)
[plotManyRegions]   plotting region 17 (out of 28)
[plotManyRegions]   plotting region 18 (out of 28)
[plotManyRegions]   plotting region 19 (out of 28)


png 
  2

In [20]:
write.csv(DMRs_filtered_by_width, paste(name, "_dmrs_top_wide.csv", sep=""), row.names = FALSE)

In [14]:
write.csv(DMRs, paste(name, "_dmrs_all.csv", sep=""), row.names = FALSE)